In [29]:
from pipeline import *
from scrape import *
import pandas as pd


data = DataPipeline()

In [30]:
query = "What is the punishment for a public servant unlawfully buying or bidding for property under Section 169 of the IPC?"

In [31]:
output = data.run_query(query)

In [32]:
output

[NodeWithScore(node=TextNode(id_='8dfe20c1-e1d1-4172-9ceb-a0c050992831', embedding=[-0.0164390039, 0.0153948329, 0.0227040313, 0.000568939489, -0.0126973903, 0.0504682772, -0.00713517, -0.000522504, 0.0114122564, -0.0328244604, 0.0115059642, 0.0191029795, 0.0247923732, -0.00242468622, -0.0057496354, 0.0103881648, 0.0120213563, -0.0026656487, 0.0332260653, -0.034537971, -0.016827222, 0.0189958848, 0.011646525, 0.00643905625, -0.0139490571, 0.00880182814, 0.0423290953, -0.0214858316, 0.000949626905, 0.00274596969, -0.0144845294, -0.00623490708, 0.000455569912, -0.0463451408, -0.0156224081, -0.0223961342, -0.00534803094, -0.00740290619, 0.00963850413, 0.00208164915, 0.00526436325, -0.0046184496, -0.0289690588, -0.0136143863, 0.00847385172, 0.0203077924, 0.000809902034, -0.0094979424, 0.0129383523, 0.024323836, 0.0242301282, 0.0115595115, -0.020575529, -0.0149798412, -0.00232763169, -0.00783128478, 0.00758362841, -0.010568887, 0.0048359856, -0.0185273457, 0.002561901, -0.000598223123, -0.0

In [5]:
output[0].text

'Category of Bare Act\nName of the Act\nYear of Promulgation\nCriminal Laws\nIndian Penal Code\n1860\nAct Number\nEnactment Date\nChapter Number\n45\n06.10.1860\n9\nChapter Title\nSub-Chapter\nLegislated by\nOf Offence By Or Relating To Public Servants\n-\nParliament of India\nWhoever, being a public servant, and being legally bound as such public servant, not to purchase or bid for certain property, purchases or bids for that property, either in his own name or in the name of another, or jointly, or in shares with others, shall be punished with simple imprisonment for a term which may extend to two years, or with fine, or with both; and the property, if purchased, shall be confiscated.\nOffence Description\nPunishment provided\nCognizable/Non-Cognizable\nPublic servant unlawfully buying or bidding for property.\xa0\xa0 \xa0\nSimple imprisonment for 2 years, or fine, or both and confiscation of property, if purchased.\nNon-Cognizable\nBailable/Non-Bailable\nTrial Court Details\nCompoun

In [6]:
output[1].text

'{\n    "title": "indian penal code ipc section 169",\n    "url": "https://www.latestlaws.com/bare-acts/central-acts-rules/ipc-section-169-public-servant-unlawfully-buying-or-bidding-for-property/",\n    "text": "\\nCategory of Bare Act\\nName of the Act\\nYear of Promulgation\\nCriminal Laws\\nIndian Penal Code\\n1860\\nAct Number\\nEnactment Date\\nChapter Number\\n45\\n06.10.1860\\n9\\nChapter Title\\nSub-Chapter\\nLegislated by\\nOf Offence By Or Relating To Public Servants\\n-\\nParliament of India\\nWhoever, being a public servant, and being legally bound as such public servant, not to purchase or bid for certain property, purchases or bids for that property, either in his own name or in the name of another, or jointly, or in shares with others, shall be punished with simple imprisonment for a term which may extend to two years, or with fine, or with both; and the property, if purchased, shall be confiscated.\\nOffence Description\\nPunishment provided\\nCognizable/Non-Cognizable

In [7]:
output[0].text

'Category of Bare Act\nName of the Act\nYear of Promulgation\nCriminal Laws\nIndian Penal Code\n1860\nAct Number\nEnactment Date\nChapter Number\n45\n06.10.1860\n9\nChapter Title\nSub-Chapter\nLegislated by\nOf Offence By Or Relating To Public Servants\n-\nParliament of India\nWhoever, being a public servant, and being legally bound as such public servant, not to purchase or bid for certain property, purchases or bids for that property, either in his own name or in the name of another, or jointly, or in shares with others, shall be punished with simple imprisonment for a term which may extend to two years, or with fine, or with both; and the property, if purchased, shall be confiscated.\nOffence Description\nPunishment provided\nCognizable/Non-Cognizable\nPublic servant unlawfully buying or bidding for property.\xa0\xa0 \xa0\nSimple imprisonment for 2 years, or fine, or both and confiscation of property, if purchased.\nNon-Cognizable\nBailable/Non-Bailable\nTrial Court Details\nCompoun

In [8]:
data.pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00184,
 'namespaces': {'': {'vector_count': 184}},
 'total_vector_count': 184}

In [9]:
index = data.pinecone_index

# Generative QnA

In [33]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=data.OPENAI_API_KEY
)

In [34]:
from langchain_pinecone import PineconeVectorStore

text_field = "url"  # the metadata field that contains our context

vector_store = PineconeVectorStore(index, embed, text_field)

In [38]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

query = "What is the punishment for a public servant unlawfully buying or bidding for property under Section 169 of the IPC?"

# completion llm
llm = ChatOpenAI(
    openai_api_key=data.OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [39]:
qa.invoke(query)

{'query': 'What is the punishment for a public servant unlawfully buying or bidding for property under Section 169 of the IPC?',
 'result': 'The punishment for a public servant unlawfully buying or bidding for property under Section 169 of the Indian Penal Code is imprisonment for a term which may extend to two years, or with fine, or with both.'}